# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/Work/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Work/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [5]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [6]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [7]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [8]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [9]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [10]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '63349b'. Skipping!
Property 'summary' already exists in node '6d5657'. Skipping!
Property 'summary' already exists in node '7b426e'. Skipping!
Property 'summary' already exists in node '45a3cb'. Skipping!
Property 'summary' already exists in node '6fc392'. Skipping!
Property 'summary' already exists in node 'b1a37b'. Skipping!
Property 'summary' already exists in node 'c158b1'. Skipping!
Property 'summary' already exists in node '8cb576'. Skipping!
Property 'summary' already exists in node '507f9d'. Skipping!
Property 'summary' already exists in node '54f824'. Skipping!
Property 'summary' already exists in node '1063e3'. Skipping!
Property 'summary' already exists in node '4547ad'. Skipping!
Property 'summary' already exists in node '83e08f'. Skipping!
Property 'summary' already exists in node '6cca1e'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '6d5657'. Skipping!
Property 'summary_embedding' already exists in node '6fc392'. Skipping!
Property 'summary_embedding' already exists in node '45a3cb'. Skipping!
Property 'summary_embedding' already exists in node 'b1a37b'. Skipping!
Property 'summary_embedding' already exists in node '7b426e'. Skipping!
Property 'summary_embedding' already exists in node '8cb576'. Skipping!
Property 'summary_embedding' already exists in node '63349b'. Skipping!
Property 'summary_embedding' already exists in node 'c158b1'. Skipping!
Property 'summary_embedding' already exists in node '6cca1e'. Skipping!
Property 'summary_embedding' already exists in node '507f9d'. Skipping!
Property 'summary_embedding' already exists in node '54f824'. Skipping!
Property 'summary_embedding' already exists in node '83e08f'. Skipping!
Property 'summary_embedding' already exists in node '4547ad'. Skipping!
Property 'summary_embedding' already exists in node '1063e3'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 478)

We can save and load our knowledge graphs as follows.

In [11]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 478)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [12]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [13]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.
##### ✅ Answer:
* Single-Hop Specific: Creates queries in which the answer requires context from a single document/chunk without the need to synthesize findings.
* Multi-Hop Specific: Creates queries in which the answer requires context from multiple documents/chunks without the need to synthesize findings.
* Multi-Hop Abstract: Creates queries in which the answer requires context from multiple documents/chunks in addition to needing to synthesize findings.

Finally, we can use our `TestSetGenerator` to generate our testset!

In [14]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,What is the significance of Title IV in relati...,"[Chapter 1 Academic Years, Academic Calendars,...","For Title IV purposes, the academic year is de...",single_hop_specifc_query_synthesizer
1,What is 34 CFR 668 3 a about in academic year ...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,"In the context of Volume 8, how does the inclu...",[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work conducted outside t...,single_hop_specifc_query_synthesizer
3,"FWS is like, what is it and how does it work w...",[Non-Term Characteristics A program that measu...,The Federal Work-Study (FWS) Program is an exc...,single_hop_specifc_query_synthesizer
4,What is Volume 8 about in relation to disburse...,[both the credit or clock hours and the weeks ...,Volume 8 discusses how the scheduled payment p...,single_hop_specifc_query_synthesizer
5,What are the program-specific academic year de...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...","According to the provided context, each eligib...",multi_hop_abstract_query_synthesizer
6,How do the disbursement timing requirements fo...,[<1-hop>\n\nboth the credit or clock hours and...,The disbursement timing requirements for Pell ...,multi_hop_abstract_query_synthesizer
7,how do payment periods and scheduled payment p...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that disbursement of Pell...,multi_hop_abstract_query_synthesizer
8,"In volume 2 and volume 8, what are the rules a...",[<1-hop>\n\nInclusion of Clinical Work in a St...,The context in volume 2 explains that clinical...,multi_hop_specific_query_synthesizer
9,How do the disbursement timing rules outlined ...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,The first context segment explains that in sub...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [15]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '9fdd45'. Skipping!
Property 'summary' already exists in node '3fd648'. Skipping!
Property 'summary' already exists in node '5ef073'. Skipping!
Property 'summary' already exists in node 'b8bc76'. Skipping!
Property 'summary' already exists in node 'ee81c4'. Skipping!
Property 'summary' already exists in node '6fd432'. Skipping!
Property 'summary' already exists in node '76b926'. Skipping!
Property 'summary' already exists in node '066702'. Skipping!
Property 'summary' already exists in node '5dcc59'. Skipping!
Property 'summary' already exists in node '15ba72'. Skipping!
Property 'summary' already exists in node 'e4d58e'. Skipping!
Property 'summary' already exists in node '7ba0fb'. Skipping!
Property 'summary' already exists in node '793173'. Skipping!
Property 'summary' already exists in node '1cff14'. Skipping!


...

Property 'summary_embedding' already exists in node '9fdd45'. Skipping!
Property 'summary_embedding' already exists in node 'b8bc76'. Skipping!
Property 'summary_embedding' already exists in node '6fd432'. Skipping!
Property 'summary_embedding' already exists in node 'ee81c4'. Skipping!
Property 'summary_embedding' already exists in node '066702'. Skipping!
Property 'summary_embedding' already exists in node '3fd648'. Skipping!
Property 'summary_embedding' already exists in node '1cff14'. Skipping!
Property 'summary_embedding' already exists in node '15ba72'. Skipping!
Property 'summary_embedding' already exists in node '5ef073'. Skipping!
Property 'summary_embedding' already exists in node '7ba0fb'. Skipping!
Property 'summary_embedding' already exists in node '793173'. Skipping!
Property 'summary_embedding' already exists in node '76b926'. Skipping!
Property 'summary_embedding' already exists in node '5dcc59'. Skipping!
Property 'summary_embedding' already exists in node 'e4d58e'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [16]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Wha is the School Participaton Divisin?,"[Chapter 1 Academic Years, Academic Calendars,...",The context does not provide a specific defini...,single_hop_specifc_query_synthesizer
1,What does 34 CFR 668.3(a) specify regarding ac...,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(a) specifies the minimum weeks of...,single_hop_specifc_query_synthesizer
2,Volume 8 is it standard or nonstandard?,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,Is the FWS program considered a term or non-te...,[Non-Term Characteristics A program that measu...,The FWS program is not subject to payment peri...,single_hop_specifc_query_synthesizer
4,How do guidelines for clinical work inclusion ...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The guidelines specify that clinical work cond...,multi_hop_abstract_query_synthesizer
5,How do guidelines and exceptions for including...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The guidelines specify that clinical work cond...,multi_hop_abstract_query_synthesizer
6,Pell TEACH FSEOG and loans how impact if more ...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that completing additiona...,multi_hop_abstract_query_synthesizer
7,academic year requirements and regulatory cita...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The academic year requirements specify that pr...,multi_hop_abstract_query_synthesizer
8,How do Volume 2 and Volume 8 relate to the inc...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Volume 2 discusses the requirements for academ...,multi_hop_specific_query_synthesizer
9,How do Appendix A and Appendix B collectively ...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Appendix B provides detailed guidance on disbu...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [ ]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [21]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [22]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [23]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500, #500 character since not using tokenizers 
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [24]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [25]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [26]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [27]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [29]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [30]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available mentioned in the context are:\n\n- Direct Subsidized Loans  \n- Direct Unsubsidized Loans  \n- Direct PLUS Loans (including student Federal PLUS Loans and parent Direct PLUS Loans for dependent students)  \n- Subsidized and Unsubsidized Federal Stafford Loans (made under the Federal Family Education Loan (FFEL) Program before July 1, 2010)  \n- Federal SLS Loans  \n- Federal PLUS Loans (part of the FFEL Program before July 1, 2010)\n\nNote: New loans are not made under the FFEL Program since July 1, 2010; currently, loans are made under the Direct Loan Program.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [31]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [32]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

##### ✅ Answer:
- `qa_evaluator`: Factual accuracy without conflicting statements.
- `labeled_helpfulness_evaluator`: Helpfulness of the response with respect to the reference answer.
- `empathy_evaluator`: Level of empathy in the response. Specifically, does it address concerns of the user specified in the query.

## LangSmith Evaluation

In [34]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'weary-nail-85' at:
https://smith.langchain.com/o/b538314a-4903-4172-841d-f92d30cdda3d/datasets/920db634-36c9-4432-8e96-acf42f8eb25e/compare?selectedSessions=ff032490-4fc9-4f79-a66d-a207461a0cc9




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Volume 2 and Volume 7 how relate to disburseme...,Based on the provided context:\n\n- Volume 2 (...,None,Volume 2 explains how the timing of disburseme...,1,1,0,8.214615,4958fdba-cce0-45fd-91f0-1e0ee2e26b28,ad9e3c72-b230-4f7f-81b5-8c1b502b3833
1,Volume 8 is about disbursments and clinical wo...,"Based on the provided context, Volume 8 discus...",None,The context explains that Volume 8 covers topi...,0,0,0,3.435337,7b6a2d90-24eb-4a83-a7b4-e49ae66f2a2d,38d80363-5e03-4208-a5d5-6d061a338c0f
2,How do Appendix A and Appendix B collectively ...,I don't know.,None,Appendix B provides detailed guidance on disbu...,0,0,0,0.879056,83aa810a-627e-45ee-85ef-e8d4fa114fb4,6c951933-145c-48cd-a7f6-23307b443fd4
3,How do Volume 2 and Volume 8 relate to the inc...,I don't know.,None,Volume 2 discusses the requirements for academ...,0,0,0,0.649311,73778ad2-6f9d-4924-bb4d-7558c83e7c55,fca9d5b7-173e-4a3a-9b26-012fdecf92ad
4,academic year requirements and regulatory cita...,The academic year requirements and regulatory ...,None,The academic year requirements specify that pr...,1,1,0,15.494843,59372c50-ea61-49e7-bc52-bbbfbdc72dea,7dd264d7-66be-463e-aae8-e8bc69fe62a6
5,Pell TEACH FSEOG and loans how impact if more ...,"Based on the provided context, if a student co...",None,The context explains that completing additiona...,1,0,0,4.092098,bd4f319b-5b3e-4419-b1c1-33136d4b5730,ffde656b-719f-44e0-ba30-bd3d9406eced
6,How do guidelines and exceptions for including...,"Based on the provided context, clinical work c...",None,The guidelines specify that clinical work cond...,1,1,0,8.035206,6fe481a6-0376-4b69-b628-f2d1d6f34dfb,24572371-a400-4191-8d24-d1821bb038d8
7,How do guidelines for clinical work inclusion ...,The guidelines specify that clinical work cond...,None,The guidelines specify that clinical work cond...,1,0,0,5.201319,32fbded9-69a5-40dd-9021-46d8465705e9,f6ffdaca-ab52-4b31-abdb-1e8182aa4e35
8,Is the FWS program considered a term or non-te...,I don't know.,None,The FWS program is not subject to payment peri...,0,0,0,0.815547,5ea7a33d-6c33-47ee-b667-7296b898367c,7f2ebf80-1655-4e21-81d1-ed370cda9c33
9,Volume 8 is it standard or nonstandard?,I don't know.,None,Inclusion of clinical work in a standard term ...,1,0,0,0.680107,eb19014c-d8df-444c-8689-8e3d5a8494a7,a8aeb2b2-2a4c-42e7-9c96-f454c2872925


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [35]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [36]:
rag_documents = docs

In [37]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?
##### ✅ Answer:
Larger chunks could mean more context retrieved in a single chunk, but could also mix context.

In [38]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?
##### ✅ Answer:
Better embeddings can improve the matching performance during context retrieval.

In [39]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [40]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [41]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [42]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

'Thank you for your question! Based on the context you’ve shared, there are a few kinds of loans available to students and their parents to help cover the cost of attendance:\n\n1. **Direct Subsidized Loans** – These loans are based on financial need and the government pays the interest while the student is in school at least half-time.\n\n2. **Direct Unsubsidized Loans** – These are available to students regardless of financial need, and interest accrues during all periods.\n\n3. **Direct PLUS Loans** – These loans are available to parents of dependent students, or to independent students themselves, to help cover the remaining cost after other aid. There is no fixed limit, but the loan cannot exceed the cost of attendance minus other financial aid received.\n\nAlso, it’s important to note that students can sometimes qualify for a combination of these loans based on their financial situation, and if a parent cannot obtain a Direct PLUS Loan, the student may be eligible for additional 

Finally, we can evaluate the new chain on the same test set!

In [43]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'proper-shoe-17' at:
https://smith.langchain.com/o/b538314a-4903-4172-841d-f92d30cdda3d/datasets/920db634-36c9-4432-8e96-acf42f8eb25e/compare?selectedSessions=35c60a77-f759-4d9e-b092-a30b5e2a5636




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Volume 2 and Volume 7 how relate to disburseme...,Thank you for your thoughtful question. I unde...,None,Volume 2 explains how the timing of disburseme...,0,0,1,4.415971,4958fdba-cce0-45fd-91f0-1e0ee2e26b28,1168c9f3-5a36-4940-b495-e547b9090dac
1,Volume 8 is about disbursments and clinical wo...,Thank you for your thoughtful question. Based ...,None,The context explains that Volume 8 covers topi...,1,1,1,3.888227,7b6a2d90-24eb-4a83-a7b4-e49ae66f2a2d,44e37a73-6d68-47ac-81b4-f7d317b20924
2,How do Appendix A and Appendix B collectively ...,Thank you for your thoughtful question. I unde...,None,Appendix B provides detailed guidance on disbu...,0,0,1,6.856460,83aa810a-627e-45ee-85ef-e8d4fa114fb4,12b7f9c0-379b-459d-922f-117ffc21fae0
3,How do Volume 2 and Volume 8 relate to the inc...,Thank you for your thoughtful question. Based ...,None,Volume 2 discusses the requirements for academ...,0,0,1,4.668004,73778ad2-6f9d-4924-bb4d-7558c83e7c55,b03d3485-f291-414a-b910-a913bdd9a0d3
4,academic year requirements and regulatory cita...,Thank you for your question—it's clear you're ...,None,The academic year requirements specify that pr...,1,1,1,4.535147,59372c50-ea61-49e7-bc52-bbbfbdc72dea,c63909f7-66d2-4c38-b516-50405996d505
5,Pell TEACH FSEOG and loans how impact if more ...,Thank you for your thoughtful question. Based ...,None,The context explains that completing additiona...,1,0,1,4.857316,bd4f319b-5b3e-4419-b1c1-33136d4b5730,08cbddec-ee91-42c6-93c4-2e664851c6cf
6,How do guidelines and exceptions for including...,Thank you for your thoughtful question. It’s c...,None,The guidelines specify that clinical work cond...,1,1,1,8.215170,6fe481a6-0376-4b69-b628-f2d1d6f34dfb,c1b0a0ba-07c1-4aff-8621-93d58e517b8f
7,How do guidelines for clinical work inclusion ...,Thank you for your thoughtful question. From t...,None,The guidelines specify that clinical work cond...,1,1,1,5.011921,32fbded9-69a5-40dd-9021-46d8465705e9,d13518e7-e79c-4bf7-ab50-671494214742
8,Is the FWS program considered a term or non-te...,Thank you for your thoughtful question. Based ...,None,The FWS program is not subject to payment peri...,1,1,1,3.694445,5ea7a33d-6c33-47ee-b667-7296b898367c,6ae131f3-226d-4768-be50-e9f2ce948553
9,Volume 8 is it standard or nonstandard?,Thank you for your question! Based on the cont...,None,Inclusion of clinical work in a standard term ...,0,0,1,2.966536,eb19014c-d8df-444c-8689-8e3d5a8494a7,428d1d36-4b41-4cfc-a875-39539d030209


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.
##### ✅ Answer:

<img src="./compare.png" alt="Alt text" width="800" height="600">

* Correctness: Stayed the same.
* Helpfulness: I think this one went up because of the increase in chunk size in conjuntion with the empathy prompt
* Empathy: Went up because we used the empathy prompt.